In [1]:
import pandas as pd
import numpy as np
import torchaudio
import jiwer
import IPython.display as ipd
import random
import torch
import re
import json


from datasets import Dataset, load_metric, ClassLabel
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2ForCTC
from transformers import TrainingArguments, Trainer


from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from IPython.display import display, HTML


In [2]:
data_path = 'data/train.tsv'
train_data = pd.read_csv(data_path, sep='\t' , header=0)
train_data = train_data.drop(columns=['client_id', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'  ])
train_data = train_data[train_data['sentence'].str.contains("A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z")==False]
train_data

,path,sentence
0,common_voice_hi_26008353.mp3,हमने उसका जन्मदिन मनाया।
1,common_voice_hi_26010468.mp3,"साउथ दिल्ली नगर निगम सख्त, शॉपिंग मॉल के बाहर ..."
2,common_voice_hi_26010469.mp3,उत्तर कोरिया ने अमेरिका को दी हमले की धमकी
3,common_voice_hi_26010470.mp3,अगले कमरे में अनेक रोमन मूर्तियाँ हैं।
4,common_voice_hi_26010471.mp3,तुम ने टॉम को कहाँ भेज दिया?
5,common_voice_hi_26010497.mp3,सर्दोयों के आने से दिन छोटे होते जाते हैं।
6,common_voice_hi_26010498.mp3,मुझे और वक़्त दो।
7,common_voice_hi_26010500.mp3,क्या सवाल है!


In [3]:
data_path = 'data/test.tsv'
test_data = pd.read_csv(data_path, sep='\t' , header=0)
test_data = test_data.drop(columns=['client_id', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'  ])
test_data = test_data[test_data['sentence'].str.contains("A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z")==False]
test_data

,path,sentence
0,common_voice_hi_25248770.mp3,तुम्हारे बहुत सारे दुशमन हैं।
1,common_voice_hi_26950127.mp3,सुष्मिता ने जिसे पहनाया था मिस यूनिवर्स का ताज...


In [4]:
def add_file_path(text):
  text = "data/clips/" + text
  return text
train_data['path'] = train_data['path'].map(lambda x: add_file_path(x))
test_data['path'] = test_data['path'].map(lambda x: add_file_path(x))


In [5]:
train_data = Dataset.from_pandas(train_data)
test_data = Dataset.from_pandas(test_data)
train_data = train_data.remove_columns(["__index_level_0__"])
test_data = test_data.remove_columns(["__index_level_0__"])


chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\�\।\'‘’|]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

train_data = train_data.map(remove_special_characters)
test_data = test_data.map(remove_special_characters)


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))
    
show_random_elements(train_data.remove_columns(["path"]), num_examples=6)    




def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = train_data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_data.column_names)
vocab_test = test_data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test_data.column_names)

vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}




vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]


with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)
    
    
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    batch["sampling_rate"] = 16_000
    batch["target_text"] = batch["sentence"]
    return batch


# The pre-trained model used 16k sampled samples
resampler = torchaudio.transforms.Resample(48_000, 16_000) # original sampling, required sampling, 
train_data = train_data.map(speech_file_to_array_fn, remove_columns=train_data.column_names)
test_data = test_data.map(speech_file_to_array_fn, remove_columns=test_data.column_names)


  0%|          | 0/8 [00:00<?, ?ex/s]

  0%|          | 0/2 [00:00<?, ?ex/s]

,sentence
0,उत्तर कोरिया ने अमेरिका को दी हमले की धमकी
1,मुझे और वक़्त दो
2,सर्दोयों के आने से दिन छोटे होते जाते हैं
3,साउथ दिल्ली नगर निगम सख्त शॉपिंग मॉल के बाहर नहीं दिखेंगे होर्डिंग
4,तुम ने टॉम को कहाँ भेज दिया
5,अगले कमरे में अनेक रोमन मूर्तियाँ हैं


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ex/s]

  0%|          | 0/2 [00:00<?, ?ex/s]

In [10]:
rand_int = random.randint(0, len(train_data))
ipd.Audio(data=np.asarray(train_data[rand_int]["speech"]), autoplay=True, rate=16000)

In [7]:
print("Target text:", train_data[rand_int]["target_text"])
print("Input array shape:", np.asarray(train_data[rand_int]["speech"]).shape)
print("Sampling rate:", train_data[rand_int]["sampling_rate"])

Target text: सर्दोयों के आने से दिन छोटे होते जाते हैं 
Input array shape: (56448,)
Sampling rate: 16000


In [12]:
rand_int = random.randint(0, len(test_data))
ipd.Audio(data=np.asarray(test_data[rand_int]["speech"]), autoplay=True, rate=16000)

In [13]:
print("Target text:", test_data[rand_int]["target_text"])
print("Input array shape:", np.asarray(train_data[rand_int]["speech"]).shape)
print("Sampling rate:", test_data[rand_int]["sampling_rate"])

Target text: सुष्मिता ने जिसे पहनाया था मिस यूनिवर्स का ताज कैंसर से हुई मौत 
Input array shape: (79872,)
Sampling rate: 16000


In [14]:
tokenizer = Wav2Vec2CTCTokenizer("vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
processor.save_pretrained("wav2vec2-large-xlsr-sinhala/")    



def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch



train_data = train_data.map(prepare_dataset, remove_columns=train_data.column_names, batch_size=8, num_proc=4, batched=True)
test_data = test_data.map(prepare_dataset, remove_columns=test_data.column_names, batch_size=8, num_proc=4, batched=True)

/home/crl/.local/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/home/crl/.local/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/home/crl/.local/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different length

In [15]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [16]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
wer_metric = load_metric("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [17]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    #gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)


model.freeze_feature_extractor()


training_args = TrainingArguments(
  output_dir="wav2vec2-large-xlsr-hindi",
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  #gradient_checkpointing=True,
  save_steps=50,
  eval_steps=50,
  logging_steps=5,
  learning_rate=1e-4,
  warmup_steps=100,
  save_total_limit=2,
)


trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=processor.feature_extractor,
)


Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_hid.bias', 'quantizer.weight_proj.weight', 'project_q.bias', 'quantizer.weight_proj.bias', 'quantizer.codevectors', 'project_hid.weight', 'project_q.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to u

In [21]:
trainer.train()

***** Running training *****
  Num examples = 8
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 30


ValueError: Unable to create tensor, you should probably activate padding with 'padding=True' to have batched tensors with the same length.